In [ ]:
#处理redis初始化从数据库中加载车辆信息以及定时将redis最新数据同步到数据库
import redis
import sched
import time
import psycopg2 as pg
import requests as req
import logging
import configparser

In [ ]:
logging.basicConfig(filename="redis-syn.log", filemode="w", format="%(asctime)s %(name)s:%(levelname)s:%(message)s", datefmt="%Y-%M-%d %H:%M:%S", level=logging.INFO)
parser = configparser.ConfigParser()
parser.read('application.ini')
logging.info('加载application.ini配置')
#获取redis配置信息
redis_host = parser.get('redis', 'host')
bus_pos_key = parser.get('redis', 'bus_pos_key')
syn_time = float(parser.get('redis', 'syn_time'))

#获取postgresql配置信息
pg_host = parser.get('postgresql', 'host')
pg_user = parser.get('postgresql', 'user')
pg_password = parser.get('postgresql', 'password')
pg_database = parser.get('postgresql', 'database')

In [ ]:
#初始化redis数据连接
def init_redis(host):
    try:
        pool = redis.ConnectionPool(host=redis_host)
        rd = redis.Redis(connection_pool=pool)
        logging.info('redis已连接')
        return rd
    except Exception as ex:
        logging.exception('redis初始化错误')

In [ ]:
#初始化db数据连接
def init_db(host, database, user_name, password):
    try:
        pgconn = pg.connect(database = database, user = user_name, password = password, host = host)
        curs = pgconn.cursor()
        logging.info('postgresql已连接')
        return pgconn, curs
    except Exception as ex:
        logging.exception('postgresql初始化错误')   

In [ ]:
def close(rd, pgconn, curs):
    if (rd != null):
        logging.info('redis已关闭')
        rd.close()
    if (curs != null):
        curs.close()
        pgconn.close()
        logging.info('postgresql已关闭')

In [ ]:
rd = init_redis(redis_host)
conn, curs = init_db(pg_host, pg_database, pg_user, pg_password)

In [ ]:
#TODO 缓存为2个 公交车基本信息 公交车当前位置信息
query_bus_pos = 'select id, ST_X(geom), ST_Y(geom), ST_M(geom), name, update_time from v_bus_pos'
query_bus = ''

In [ ]:
def init_cache_from_db():
    logging.info('从postgresql中加载车辆位置数据')
    try:
        curs.execute(sql)
        data = curs.fetchall()
        for d in data:
            id = d[0]
            x = d[1]
            y = d[2]
            m = d[3]
            name = d[4]
            update_time = d[5]
            v = str(id) + '|' + str(x) + '|' + str(y) + '|' + str(m) + '|' + name + '|' + str(update_time)
            #TODO 考虑各公交公司车辆以单位分开存放
            rd.hset(name= bus_pos_key, key= id, value= v)
        logging.info('从postgresql中加载车辆数据已成功')
    except Exception as ex:
         logging.exception('初始化缓存出错')   

In [ ]:
#首先重新加载缓存
init_cache_from_db()

In [ ]:
rest_url = 'http://localhost:8080/websocket/sendMessage?message='

In [ ]:
for key in rd.keys():
    bus = rd.hgetall('buspos')
    for k in bus:
        req.post(rest_url + bus[k].decode())

In [ ]:
#初始化定时任务
s = sched.scheduler(time.time, time.sleep)

In [ ]:
#缓存中新的数据同步到数据库  TODO
def syn_to_db(time_rate):
    print("From print_time", time.time())
    s.enter(time_rate,0,syn_to_db,(time_rate,))

In [ ]:
#每隔五分钟执行一次
if not conn is None:
    s.enter(0,0, syn_to_db, (syn_time,))
    s.run()